In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ConvolutionalVAE(nn.Module):
    def __init__(
        self,
        input_dim: int,
        n_channels: int,
        conv_dim: int,
        latent_dim: int,
        batch_size: int,s
    ):
        super().__init__()
        self.input_dim = input_dim
        self.n_channels = n_channels
        self.conv_dim = conv_dim
        self.latent_dim = latent_dim
        self.batch_size = batch_size
        
        self.encoder = nn.Sequential(
            nn.Conv2d(n_channels, conv_dim, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(conv_dim),
            nn.Conv2d(conv_dim, 2 * conv_dim, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(2 * conv_dim),
            nn.Conv2d(2 * conv_dim, 4 * conv_dim, kernel_size=3, stride=2, padding=1), 
            nn.ReLU(),
            nn.BatchNorm2d(4 * conv_dim),
            nn.Flatten(),
            nn.Linear(
                conv_dim * 4 * self.input_dim[1] // 8 * self.input_dim[2] // 8, 2 * latent_dim
            ),
        )
        

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 4 * conv_dim * self.input_dim[1] // 8 * self.input_dim[2] // 8),
            nn.Unflatten(1, (4 * conv_dim, self.input_dim[1] // 8, self.input_dim[2] // 8)),  # Reshape to (batch, channels, height, width)
            nn.ConvTranspose2d(4 * conv_dim, 2 * conv_dim, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(2 * conv_dim),
            nn.ConvTranspose2d(2 * conv_dim, conv_dim, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(conv_dim),
            nn.ConvTranspose2d(conv_dim, n_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )
        
    def sample(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        encoded = self.encoder(x)
        mu, logvar = encoded.split(self.latent_dim, dim=1)
        z = self.sample(mu, logvar)
        return self.decoder(z)

In [45]:
# Checking that dimenionality is correct
vae = ConvolutionalVAE(
    input_dim=(3, 32, 32),
    n_channels=3,
    conv_dim=96,
    latent_dim=128,
    batch_size=4,
)

random_data = torch.randn(4, 3, 32, 32)
x_hat = vae(random_data)

print("x_hat shape", x_hat.shape)

x_hat shape torch.Size([4, 3, 32, 32])
